In [1]:
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

c:\Users\SALAH\anaconda3\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:07<00:00, 3565651.27it/s]


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 29515/29515 [00:00<00:00, 437702.22it/s]


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:01<00:00, 2928825.55it/s]


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 5148/5148 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [4]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [14]:
batch_size = 64                                                # Define the batch size for data loading

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)  # Create a DataLoader for training data, 
                                                                    # specifying the training_data and batch_size
test_dataloader = DataLoader(test_data, batch_size=batch_size)      # Create a DataLoader for test data, 
                                                                    # specifying the test_data and batch_size

for X, y in test_dataloader:                                    # Iterate through the batches of the test dataloader
    print(f"Shape of X [N, C, H, W]: {X.shape}")               # Print the shape of the input data batch
                                                                # [N, C, H, W] corresponds to [batch_size, channels, height, width]
    print(f"Shape of y: {y.shape} {y.dtype}")                 # Print the shape and data type of the labels batch
    break                                                       # Break the loop after processing the first batch


Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [7]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):                         # Define a class named NeuralNetwork which inherits from nn.Module
    def __init__(self):                                 # Define the constructor method for the class
        super().__init__()                              # Call the constructor of the superclass (nn.Module)
        self.flatten = nn.Flatten()                      # Create a flatten layer to flatten the input
        self.linear_relu_stack = nn.Sequential(          # Create a sequential container for linear layers and ReLU activations
            nn.Linear(28*28, 512),                       # Define a linear layer with input size 28*28 and output size 512
            nn.ReLU(),                                   # Apply ReLU activation function
            nn.Linear(512, 512),                         # Define another linear layer with input and output size 512
            nn.ReLU(),                                   # Apply ReLU activation function
            nn.Linear(512, 10)                           # Define the final linear layer with input size 512 and output size 10
        )

    def forward(self, x):                                # Define the forward pass method
        x = self.flatten(x)                              # Flatten the input tensor
        logits = self.linear_relu_stack(x)               # Pass the flattened tensor through the sequential layers
        return logits                                    # Return the output logits

model = NeuralNetwork().to(device)                      # Instantiate the NeuralNetwork class and move it to the specified device
print(model)                                             # Print the architecture of the model


Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
loss_fn = nn.CrossEntropyLoss()                          # Define the loss function as CrossEntropyLoss
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) # Define the optimizer as Stochastic Gradient Descent (SGD)
                                                          # with a learning rate of 1e-3, and specify that it should
                                                          # optimize the parameters of the model


In [10]:
def train(dataloader, model, loss_fn, optimizer):         # Define a function named train which takes a dataloader,
                                                          # a model, a loss function, and an optimizer as input
    size = len(dataloader.dataset)                        # Get the total size of the dataset
    model.train()                                         # Set the model in training mode
    for batch, (X, y) in enumerate(dataloader):           # Iterate over batches of data in the dataloader
        X, y = X.to(device), y.to(device)                # Move the input data to the appropriate device

        # Compute prediction error
        pred = model(X)                                   # Forward pass: compute the predictions of the model
        loss = loss_fn(pred, y)                           # Compute the loss between the predictions and the ground truth labels

        # Backpropagation
        loss.backward()                                   # Backward pass: compute gradients of the loss with respect to model parameters
        optimizer.step()                                  # Update model parameters using the gradients and the optimizer
        optimizer.zero_grad()                             # Zero the gradients to prevent them from accumulating

        if batch % 100 == 0:                              # Print the loss and current progress every 100 batches
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [16]:
def test(dataloader, model, loss_fn):                  # Define a function named test which takes a dataloader,
                                                      # a model, and loss function, as input
    size = len(dataloader.dataset)                    # Get the total size of the dataset
    num_batches = len(dataloader)                      # Get the total number of batches in the dataloader
    model.eval()                                      # Set the model in evaluation mode
    test_loss, correct = 0, 0                         # Initialize variables to keep track of test loss and correct predictions
    with torch.no_grad():                             # Turn off gradient tracking during inference
        for X, y in dataloader:                       # Iterate over batches of data in the dataloader
            X, y = X.to(device), y.to(device)        # Move the input data and labels to the appropriate device
            pred = model(X)                           # Forward pass: Compute predictions of the model for the input data
            test_loss += loss_fn(pred, y).item()      # Compute the loss between the predictions and the actual labels and accumulate it
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()  # Compute the number of correct predictions

    test_loss /= num_batches                          # Compute the average test loss across all batches
    correct /= size                                   # Compute the accuracy by dividing the number of correct predictions by the total size of the dataset
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")  # Print the test accuracy and average loss


In [17]:
epochs = 5                                        # Define the number of epochs for training
for t in range(epochs):                           # Iterate over each epoch
    print(f"Epoch {t+1}\n-------------------------------")  # Print the current epoch number
    train(train_dataloader, model, loss_fn, optimizer)  # Train the model using the train_dataloader, loss function, and optimizer
    test(test_dataloader, model, loss_fn)          # Evaluate the model using the test_dataloader and loss function
print("Done!")                                    # Print "Done!" when training and evaluation are complete


Epoch 1
-------------------------------
loss: 1.593058  [   64/60000]
loss: 1.554096  [ 6464/60000]
loss: 1.409743  [12864/60000]
loss: 1.474356  [19264/60000]
loss: 1.385597  [25664/60000]
loss: 1.371961  [32064/60000]
loss: 1.383271  [38464/60000]
loss: 1.306121  [44864/60000]
loss: 1.336372  [51264/60000]
loss: 1.239598  [57664/60000]
Test Error: 
 Accuracy: 61.5%, Avg loss: 1.267142 

Epoch 2
-------------------------------
loss: 1.343862  [   64/60000]
loss: 1.321099  [ 6464/60000]
loss: 1.162178  [12864/60000]
loss: 1.261115  [19264/60000]
loss: 1.160411  [25664/60000]
loss: 1.177934  [32064/60000]
loss: 1.197412  [38464/60000]
loss: 1.132840  [44864/60000]
loss: 1.163508  [51264/60000]
loss: 1.080834  [57664/60000]
Test Error: 
 Accuracy: 63.8%, Avg loss: 1.103531 

Epoch 3
-------------------------------
loss: 1.175179  [   64/60000]
loss: 1.169537  [ 6464/60000]
loss: 0.994973  [12864/60000]
loss: 1.123303  [19264/60000]
loss: 1.013951  [25664/60000]
loss: 1.043256  [32064/600

In [18]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [19]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [20]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
